In [ ]:
import pandas as pd
import numpy as np
import re
import time

import requests as rq
import bs4 as bs4
import json
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Coleta

In [ ]:
df = pd.read_json("/content/drive/My Drive/Projetos/Rec_carros/Scraper_busca/Links_carros/parsed_links.json", lines= "True")
df['modelo'] = df['link'].map(lambda x: x.split("/")[-3])
df.head()

,link,modelo
0,https://www.shopcar.com.br/veiculos/vw-volkswa...,polo-hatch-highline-1-0-12v-tsi
1,https://www.shopcar.com.br/veiculos/vw-volkswa...,polo-hatch-highline-1-0-12v-tsi
2,https://www.shopcar.com.br/veiculos/vw-volkswa...,polo-hatch-comfortline-1-0-12v-tsi
3,https://www.shopcar.com.br/veiculos/vw-volkswa...,polo-hatch-comfortline-1-0-12v-tsi
4,https://www.shopcar.com.br/veiculos/vw-volkswa...,polo-hatch-comfortline-1-0-12v-tsi


In [ ]:
for link in df["link"]:
    response = rq.get(link)
    
    link_name = link.split("/")[-3]
    hash = link.split("/")[-1]
    
    with open("/content/drive/My Drive/Projetos/Rec_carros/Scraper_carro/{}_{}.html".format(link_name, hash), 'w+') as output:
        output.write(response.text)

#Processamento de Dados

In [ ]:
for link in df["link"][:1]:

  modelo = link.split("/")[-3]
  hash = link.split("/")[-1]
  
  with open("/content/drive/My Drive/Projetos/Rec_carros/Scraper_carro/{}_{}.html".format(modelo, hash), 'r+') as inp:
    page_html = inp.read()

    parsed = bs4.BeautifulSoup(page_html)

    # Coletar Ano, Cor e KM

    especs = parsed.find("ul", "especs")
    
    especs1 = especs.span.extract()
    especs2 = especs.span.extract()
    especs3 = especs.span.extract()

    ano = especs1.get_text()
    cor = especs2.get_text()
    km = especs3.get_text()

    # Coletar Opcionais

    extras = parsed.find("ul", "opcionais")
    if extras is None:
      opcionais = []
    else:
      extras1 = extras.find_all('li')
      
      opcionais = []
      for e in extras1:
        extras.span.extract()
        i = extras.li.extract()
        item = i.get_text()
        opcionais.append(item)

    # Coletar Valor do Veículo

    preco = parsed.find("div", "barra-preco")
    for string in preco.stripped_strings:
      valor = string
    
    #Salvando os dados

    with open("/content/drive/My Drive/Projetos/Rec_carros/Scraper_carro/Data/parsed_carros.json", 'a+') as output:
      data = {"id": hash, "modelo": modelo, "ano": ano, "valor": valor, "cor": cor, "km": km, "opcionais": opcionais, "link": link}
      output.write("{}\n".format(json.dumps(data)))

In [ ]:
data = pd.read_json("/content/drive/My Drive/Projetos/Rec_carros/Scraper_carro/Data/parsed_carros.json", lines="True")
data.shape

(71, 8)

In [ ]:
data.to_csv("/content/drive/My Drive/Projetos/Rec_carros/Scraper_carro/Data/data.csv", index= False)
#data.to_excel("/content/drive/My Drive/Projetos/Rec_carros/Scraper_carro/Data/data.xls")